In [1]:
from dotenv import load_dotenv
from openai import OpenAI

load_dotenv("env_variables.env")
client = OpenAI()

In [2]:
from promptrefiner import AbstractLLM, PromptTrackerClass, OpenaiCommunicator

In [3]:
###
class LlamaCPPModel(AbstractLLM):
    def __init__(self, base_url, api_key, temperature=0.1, max_tokens=200):
        super().__init__()
        self.temperature = temperature
        self.max_tokens = max_tokens
        from openai import OpenAI
        self.client = OpenAI(base_url=base_url, api_key=api_key)
        
    def predict(self, input_text, system_prompt):
        response = self.client.chat.completions.create(
        model="mistral",
        messages=[
            {"role": "system", "content": system_prompt},  # Update this as per your needs
            {"role": "user", "content": input_text}
        ],
        temperature=self.temperature,
        max_tokens=self.max_tokens,
    )
        llm_response = response.choices[0].message.content
        return llm_response

In [4]:
llamamodel = LlamaCPPModel(base_url="http://localhost:8000/v1", api_key="sk-xxx", temperature=0.1, max_tokens=400)

In [5]:
###
input_evaluation_1 = """In an era where the digital expanse collided with the realm of human creativity, two figures stood at the forefront. Ada, with her prophetic vision, laid the groundwork for computational logic, while Banksy, shrouded in anonymity, painted the streets with social commentary. Amidst this, the debate over AI ethics, spearheaded by figures like Bostrom, questioned the trajectory of our technological companions. This period, marked by innovation and introspection, challenges us to ponder the relationship between creation and creator."""

input_evaluation_2 = """As the digital revolution accelerated, two contrasting visions emerged. Musk's endeavors to colonize Mars signified humanity's boundless ambition, while Hawking's warnings about AI highlighted the existential risks of unbridled technological advancement. Amid these towering ambitions and cautions, the resurgence of environmentalism, led by figures like Thunberg, reminded us of our responsibilities to Earth. This dichotomy between reaching for the stars and preserving our home planet defines the modern dilemma."""

output_evaluation_1 = """
["Ada Lovelace: Computational logic", "Banksy: Social commentary through art", "Nick Bostrom: AI ethics", "Digital era: Innovation and introspection", "AI ethics: The debate over the moral implications of artificial intelligence"]
"""

output_evaluation_2 = """
["Elon Musk: Colonization of Mars", "Stephen Hawking: Warnings about AI", "Greta Thunberg: Environmentalism", "Digital revolution: Technological advancement and existential risks", "Modern dilemma: Balancing ambition with environmental responsibility"]
"""


In [6]:
input_evaluations = [input_evaluation_1, input_evaluation_2]
output_evaluations = [output_evaluation_1, output_evaluation_2]

In [7]:
###
init_sys_prompt = """You are an AI that receives an input text. Your task is to output a pythoning string where every strings is the name of a person with what they are associated with"""
promptTracker = PromptTrackerClass(init_system_prompt = init_sys_prompt)
promptTracker.add_evaluation_examples(input_evaluations, output_evaluations)
#promptTracker.add_evaluation_examples(input_evaluation_2, output_evaluation_2)

In [8]:
promptTracker.run_initial_prompt(llm_model=llamamodel)

In [9]:
print(promptTracker.llm_responses[0][0])

# Define the names and their associations
ada = "Ada, the visionary mathematician and computer scientist, laid the groundwork for computational logic."
bostrom = "Nick Bostrom, the philosopher, spearheaded the debate over AI ethics, questioning the trajectory of our technological companions."
banksy = "Banksy, the anonymous artist, painted the streets with social commentary in the digital and physical world."
# Print out the Python strings
print(ada)
print(bostrom)
print(banksy)



In [10]:
print(promptTracker.llm_responses[0][1])

```python
persons = {
    "Musk, Elon": "space_exploration",
    "Hawking, Stephen": "ai_ethics",
    "Thunberg, Greta": "environmentalism"
}
print(f"Elon Musk is known for {persons['Musk, Elon']}")
print(f"Stephen Hawking is known for {persons['Hawking, Stephen']}")
print(f"Greta Thunberg is known for {persons['Thunberg, Greta']}")
```
This Python code creates a dictionary called `persons` where each key-value pair represents a person and their associated field. The `print` statements then display the person's name and their field of expertise.


In [11]:
openai_communicator = OpenaiCommunicator(client=client, openai_model_code="gpt-4-0125-preview")

In [12]:
openai_communicator.refine_system_prompt(prompttracker=promptTracker, llm_model=llamamodel, number_of_iterations=3)

In [13]:
import textwrap

In [14]:
print(textwrap.fill(promptTracker.llm_system_prompts[0], width=100))

You are an AI that receives an input text. Your task is to output a pythoning string where every
strings is the name of a person with what they are associated with


In [20]:
print(textwrap.fill(promptTracker.llm_responses[0][0], width=100))

# Define the names and their associations ada = "Ada, the visionary mathematician and computer
scientist, laid the groundwork for computational logic." bostrom = "Nick Bostrom, the philosopher,
spearheaded the debate over AI ethics, questioning the trajectory of our technological companions."
banksy = "Banksy, the anonymous artist, painted the streets with social commentary in the digital
and physical world." # Print out the Python strings print(ada) print(bostrom) print(banksy)


In [21]:
print(textwrap.fill(promptTracker.llm_responses[0][1], width=100))

```python persons = {     "Musk, Elon": "space_exploration",     "Hawking, Stephen": "ai_ethics",
"Thunberg, Greta": "environmentalism" } print(f"Elon Musk is known for {persons['Musk, Elon']}")
print(f"Stephen Hawking is known for {persons['Hawking, Stephen']}") print(f"Greta Thunberg is known
for {persons['Thunberg, Greta']}") ``` This Python code creates a dictionary called `persons` where
each key-value pair represents a person and their associated field. The `print` statements then
display the person's name and their field of expertise.


In [17]:
print(textwrap.fill(promptTracker.llm_system_prompts[1], width=100))

You are an AI designed to analyze text and extract key figures along with their primary associations
or contributions. Your output should be a list of strings, each string representing a person or a
concept followed by a colon and a brief description of their association or contribution. The
description should be concise and to the point. Avoid generating code or detailed explanations.
Focus on identifying the most relevant figures and concepts mentioned in the text and their succinct
associations. Here are three examples to guide your behavior:  Example 1: Input: "In the digital
age, Ada Lovelace's work laid the groundwork for modern computing, while Banksy's art offers social
commentary." Output: ["Ada Lovelace: Modern computing", "Banksy: Social commentary"]  Example 2:
Input: "Elon Musk's ambition to colonize Mars contrasts with Stephen Hawking's warnings about AI,
highlighting a modern dilemma." Output: ["Elon Musk: Colonization of Mars", "Stephen Hawking:
Warnings about AI", "Mod

In [18]:
print(textwrap.fill(promptTracker.llm_responses[1][0], width=100))

["Ada: Prophetic vision, groundwork for computational logic", "Banksy: Social commentary through
art", "AI ethics: Questioning the trajectory of technological companions", "Figures like Bostrom:
Leading the debate"]


In [19]:
print(textwrap.fill(promptTracker.llm_responses[1][1], width=100))

["Elon Musk: Colonization of Mars", "Stephen Hawking: Warnings about AI", "Greta Thunberg:
Environmentalism", "Modern dilemma: Ambition vs. responsibility"]
